In [2]:
#general imports
import sys
sys.path.append(r'C:\Users\Robert Lees\Documents\Code\Vape\suite2p_etc')
sys.path.append('..')

import numpy as np
import os
import time

In [10]:
#notebook specific imports
from utils.gsheets_importer import gsheet2df, correct_behaviour_df, split_df, path_conversion, path_finder
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import *
from utils.paq2py import *
from utils.parse_metadata import *
import tifffile
import copy
import math

import suite2p
print(suite2p.__path__)
from suite2p.run_s2p import run_s2p
from settings import ops

import matplotlib.pyplot as plt

['C:\\ProgramData\\Anaconda3\\lib\\site-packages\\suite2p']


In [19]:
# read g sheets to populate some lists for processing stim artifact and running suite2p

sheet_ID = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
SHEET_NAME = '2019-06-25_RL048'
df = gsheet2df(sheet_ID, HEADER_ROW=2, SHEET_NAME=SHEET_NAME)

# at this point we have lots of files that could be whisker stim or photostim, need to find out which is which

for_processing = split_df(df, 's2p_me') # only files with TRUE in suite2p_me column

if not for_processing.shape[0]:
    raise Exception('ERROR: no files set for processing')

stim = for_processing.loc[:,'stim'] # find out what stims have been carried out
photostim_idx = [i for i,stim in enumerate(stim) if stim=='p'] # row indices of all photostim exps (for artifact removal)
whisker_stim_idx = [i for i,stim in enumerate(stim) if stim=='w'] # '' for whisker stim (no artifact removal)

if ( len(photostim_idx) + len(whisker_stim_idx) ) != stim.shape[0]:
    raise Exception('ERROR: stim type is not defined for some files')

tiff_paths = for_processing.loc[:,'tiff_path']
naparm_paths = for_processing.loc[:,'naparm_path']

# if not all(tiff_paths) or not all(naparm_paths):
#     raise Exception('ERROR: missing tiff or naparm paths')
    
packerstation_path = r"P:" # the path to PackerStation on the local machine
# TODO: update this to path_finder rather than conversion, to increase failsafe at this point
tiffs_pstation = path_conversion(tiff_paths, packerstation_path) # convert paths (from Packer1 or PackerStation) to local PackerStation paths
# naparm_pstation = path_conversion(naparm_paths, packerstation_path)

print(tiffs_pstation, naparm_pstation)

['P:sarmstrong\\Data\\2019-06-25\\RL048\\2019-06-24_RL048_t-005', 'P:sarmstrong\\Data\\2019-06-25\\RL048\\2019-06-24_RL048_t-006\\MPTIFF'] ['P:sarmstrong\\Data\\2019-06-25\\RL048\\2019-06-24_RL048_NAPARM\\2019-06-25_RL048_NAPARM_003', 'P:sarmstrong\\Data\\2019-06-25\\RL048\\2019-06-24_RL048_NAPARM\\2019-06-25_RL048_NAPARM_004']


In [14]:
pv_values, naparm_xml, naparm_gpl = getMetadata(tiffs_pstation, naparm_pstation)

P:sarmstrong\Data\2019-06-26\2019-06-26_t-005\MPTIFF
P:sarmstrong\Data\2019-06-26\2019-06-26_t-005\2019-06-26_t-005_BACKUP.xml
P:sarmstrong\Data\2019-06-25\RL048\2019-06-24_RL048_NAPARM\2019-06-25_RL048_NAPARM_003\2019-06-25_RL048_NAPARM_003.xml
P:sarmstrong\Data\2019-06-25\RL048\2019-06-24_RL048_NAPARM\2019-06-25_RL048_NAPARM_003\2019-06-25_RL048_NAPARM_003.gpl
Number of frames: 27217 
Number of planes: 1
Frames per second: 24.519006361382644
Frame averaging: 1
Size (x): 1020
Size (y): 626
Imaging laser power: 248.717948717949
Pixel size (x,y): 1.31338993281181 1.31338993281181
Number of groups: 4 
Number of sequence reps: 10 
Number of shots: 1 
Numbers of trials: 100 
Inter-point delay: 5
Repetitions: 1
Single stim dur (ms): 10


In [15]:
# obtain list of tiffs to run stim removal on

tiff_lists = []

# need to find single multi-page TIFF or many TIFFs/MPTIFFs

for tiff in tiffs_pstation:
    print(tiff)
    
    if '.tif' not in tiff or '.tiff' not in tiff: # if the path is not directly a TIFF, it may be folder with MPTIFF or multiple TIFFs/MPTIFFs
        
        items = os.listdir(tiff)
        newlist = []
        
        for name in items:
            if name.endswith(".tiff") or name.endswith(".tif"):
                filename = os.path.join(tiff, name)
                newlist.append(filename)
        tiff_lists.append(newlist)
        
    else: # if provided path directs to TIFF file, make sure it is only one in folder
        
        tiff_count = 0
        
        parent_dir = os.path.dirname(tiff)
        items = os.listdir(parent_dir)
                
        for name in items:
            if name.endswith(".tiff") or name.endswith(".tif"):
                tiff_count += 1
        
        if tiff_count > 1:
            raise Exception('ERROR: make sure large, single TIFF files are in their own folder')
        else: 
            tiff_lists.append(tiff)
            
photostim_tiffs = [tiff_lists[i] for i in photostim_idx] # only run artifact removal-specific code on photostim exps

P:sarmstrong\Data\2019-06-26\2019-06-26_t-005\MPTIFF


In [17]:
# important: data paths must be lists even if only one element
# can also only run on specified tiffs

user_batch_size = 200 # number of frames to be processed at once (i.e. registered)

db = []

for i,tiff_list in enumerate(tiff_lists):
    print(tiff_list[0])
    
    if tiff_list in photostim_tiffs: # photostim experiments should have tiffs processed with artifact removed
        print('Photostim experiment')
        umbrella_folder = os.path.dirname(tiff_list[0])
#         folder_name = path_finder(umbrella_folder, 'Artifact_removed',  is_folder=True) # find new folder containing artifact-removed data
        folder_name = umbrella_folder
    else: # whisker stim experiments
        print('Whisker stim experiment')
        umbrella_folder = os.path.dirname(tiff_list[0])
#         folder_name = path_finder(umbrella_folder, 'Artifact_removed',  is_folder=True) # find new folder containing artifact-removed data
        folder_name = umbrella_folder
        
#     tiff_list = []
    
#     for file in os.listdir(folder_name):
#             if '.ome' not in file and '.tif' in file:
#                 tiff_list.append(file) 
    
#     tiff_list = sorted(tiff_list)
    
    fps = pv_values[i][0]
    n_planes = pv_values[i][5]
    pixel_x = pv_values[i][7]
    pixel_y = pv_values[i][8]
    frame_size = pv_values[i][2:4]
    num_pixels = frame_size[0]*frame_size[1]
    
    sampling_rate = fps/n_planes
    diameter_x = 13/pixel_x
    diameter_y = 13/pixel_y
    diameter = int(diameter_x), int(diameter_y)
    batch_size = user_batch_size * (262144 / num_pixels) # larger frames will be more RAM intensive, scale user batch size based on num pixels in 512x512 images
      
    db.append({ 'data_path' : [folder_name], 
#               'tiff_list' : tiff_list,
              'fs' : float(sampling_rate),
              'diameter' : diameter, 
              'batch_size' : int(batch_size), 
              'nimg_init' : int(batch_size),
              'nplanes' : n_planes
              })
    
db

P:sarmstrong\Data\2019-06-26\2019-06-26_t-005\MPTIFF\2019-06-26_t-005_Cycle00001_Ch3.tif
Whisker stim experiment


[{'data_path': ['P:sarmstrong\\Data\\2019-06-26\\2019-06-26_t-005\\MPTIFF'],
  'fs': 24.519006361382644,
  'diameter': (9, 9),
  'batch_size': 82,
  'nimg_init': 82,
  'nplanes': 1}]

In [18]:
t1 = time.time()

for dbi in db:
    opsEnd = run_s2p(ops=ops,db=dbi)
    
t2 = time.time()
t2 - t1

Found 1 tifs


C:\ProgramData\Anaconda3\lib\site-packages\skimage\external\tifffile\tifffile.py:1635: UserWarning: can not reshape (2000, 626, 1020) to (27217, 626, 1020)
  warnings.warn(str(e))


27217
time 3241.7584. Wrote tifs to binaries for 1 planes
C:\BIN\suite2p\plane0\data_raw.bin
computed reference frame for registration
registered 410/27217 frames in time 104.37
registered 820/27217 frames in time 207.28
registered 1230/27217 frames in time 310.51
registered 1640/27217 frames in time 413.71
registered 2050/27217 frames in time 517.29
registered 2460/27217 frames in time 620.10
registered 2870/27217 frames in time 723.73
registered 3280/27217 frames in time 826.88
registered 3690/27217 frames in time 931.15
registered 4100/27217 frames in time 1034.85
registered 4510/27217 frames in time 1138.51
registered 4920/27217 frames in time 1241.89
registered 5330/27217 frames in time 1345.61
registered 5740/27217 frames in time 1449.00
registered 6150/27217 frames in time 1552.48
registered 6560/27217 frames in time 1656.16
registered 6970/27217 frames in time 1759.91
registered 7380/27217 frames in time 1863.21
registered 7790/27217 frames in time 1967.46
registered 8200/27217

11668.217411518097